In [23]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
from xhistogram.xarray import histogram
import os
import time
from matplotlib.colors import LogNorm
import dask

In [24]:
listOfDir = os.listdir('/Data2/SubX/')
listOfDir = listOfDir[0:-1] # drop 'CWB
listOfDir

['ESRL', 'NASA', 'EMC', 'RSMAS', 'ECCC', 'NRL', 'CESM', 'NCEP']

In [25]:
subfolder_to_try = ['FIMr1p1','GEOS_V2p1','GEFS','CCSM4','GEPS5','GEPS6','hindcast','NESM','46LCESM1', '30LCAM5','CFSv2']

# the list above will look for the subfolder in the folder. I have 11 possibilities, one for each model. In particular

In [26]:
for ild in listOfDir:
    for isb in subfolder_to_try:
        if isb in os.listdir('/Data2/SubX/'+ild+'/'):
            print(ild+'/'+isb)


ESRL/FIMr1p1
NASA/GEOS_V2p1
EMC/GEFS
RSMAS/CCSM4
ECCC/GEPS5
ECCC/GEPS6
ECCC/hindcast
NRL/NESM
CESM/46LCESM1
CESM/30LCAM5
NCEP/CFSv2


## each of this subfloders are organized differently (in fact the third ECCC doesn't have a name for the submodel, but directly the folder hindcast. I assume that is the GEM model)

## Unfortunately I cannot use a general loop to treat each model, but I have to write a separate script for each of them

## the name of the variables *should* be the same: cape, tas_2m, tdps_2m, wap_500, pr_sfc

In [7]:
variable_names = ['cape', 'tas_2m', 'tdps_2m', 'wap_500', 'pr_sfc']

# RSMAS/CCSM4/


In [27]:
for ild in listOfDir[3:4]:
    for isb in subfolder_to_try:
        if isb in os.listdir('/Data2/SubX/'+ild+'/'):
            subfolder_dir = ild+'/'+isb
            print(subfolder_dir)
            print('/Data2/SubX/'+subfolder_dir)
            file_list = os.listdir('/Data2/SubX/'+subfolder_dir)
#             file_list1 = [ifl if ifl[-4:]!='.zip' else print('skip') for ifl in file_list]
            print(file_list)


RSMAS/CCSM4
/Data2/SubX/RSMAS/CCSM4
['tas_2m', 'hfls', 'hfss', 'huss', 'mrso', 'pr', 'psl', 'rad', 'rlut', 'snc', 'snwd', 'stx', 'sty', 'swe', 'ta', 'tasmax_2m', 'tasmin_2m', 'ts', 'ua', 'uas', 'va', 'vas', 'wap', 'zg', 'ua_200', 'ua_850', 'va_200', 'va_850', 'zg_200', 'zg_500', 'forecast', 'documentation.txt', 'climatology', 'cape', 'cape_sfc', 'hfls_sfc', 'hfss_sfc.csh', 'huss_850', 'mrso_sfc', 'psl_sfc', 'rad_sfc', 'sic', 'sic_sfc', 'snc_sfc', 'snwd_sfc', 'stx_sfc', 'sty_sfc', 'swe_sfc', 'ua_100', 'uas_10m', 'va_100', 'vas_10m', 'wap_500', 'RMM', 'pentad_climatology', 'CCSM3_NMME', 'NMME', 'ROMI']


In [28]:
for ild in listOfDir[3:4]:
    for isb in subfolder_to_try:
        if isb in os.listdir('/Data2/SubX/'+ild+'/'):
            subfolder_dir = ild+'/'+isb+'/'
            for ivar in variable_names:
                print(ivar)
                if ivar in os.listdir('/Data2/SubX/'+subfolder_dir):
                    var_subfolder_dir = '/Data2/SubX/'+subfolder_dir+ivar+'/'
                    file_list = os.listdir(var_subfolder_dir)
                    print(file_list)


tas_2m
['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
tdps_2m
wap_500
['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
pr
['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']


In [29]:
!ls /Data2/SubX/RSMAS/CCSM4/

cape		   huss		       rad_sfc	 stx	    ts	     va_850
cape_sfc	   huss_850	       rlut	 stx_sfc    ua	     vas
CCSM3_NMME	   mrso		       RMM	 sty	    ua_100   vas_10m
climatology	   mrso_sfc	       ROMI	 sty_sfc    ua_200   wap
documentation.txt  NMME		       sic	 swe	    ua_850   wap_500
forecast	   pentad_climatology  sic_sfc	 swe_sfc    uas      zg
hfls		   pr		       snc	 ta	    uas_10m  zg_200
hfls_sfc	   psl		       snc_sfc	 tas_2m     va	     zg_500
hfss		   psl_sfc	       snwd	 tasmax_2m  va_100
hfss_sfc.csh	   rad		       snwd_sfc  tasmin_2m  va_200


In [30]:
variable_names = [ 'tas_2m', 'tdps_2m', 'wap_500', 'pr']

In [32]:
# ds1 = xr.open_dataset('/Data2/SubX/RSMAS/CCSM4/pr/2000/01/pr_sfc_CCSM4_28jan2000_00z_d01_d45_m03.nc')
# ds1

In [ ]:
from glob import glob
def pre_process_func(ds,strM):
    ds.coords['TIME'] = ds.indexes['TIME'].to_datetimeindex()

    ds = ds.rename({'TIME':'time', 'LAT':'lat', 'LON':'lon'})
    ds = ds.sel(lat=slice(23,50), lon=slice(230,300))
    ds.coords['S'] = ds.time[0].values
    ds.coords['S'] = ds.S - np.timedelta64(12,'h') 
    ds.coords['M'] = np.int(strM)
    ds.coords['time'] = ds.time-ds.S
    ds = ds.expand_dims(['S','M'])
    return ds
def read_netcdfs(files, dim, transform_func=None):
#     print(files)
    def process_one_path(path):
#         print(path)
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path, chunks={}) as ds:
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds, path[-5:-3])
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
#             print(ds)
#             ds.load()
            return ds

    paths = sorted(glob(files))
#     print(paths)
    datasets = []
    for p in paths:
#         print(p)
        if (p.split('/')[6]=='2016')&(p.split('/')[7]=='09'):
            print(p)
        else:
            datasets.append(process_one_path(p))
#     datasets = [process_one_path(p) for p in paths]
    
    combined = xr.combine_by_coords(datasets)
    combined = combined.chunk({'S':4,'M':-1})
    return combined



var_l=[]
for ivar in variable_names:
    print(ivar)
    if ivar in os.listdir('/Data2/SubX/RSMAS/CCSM4/'):    
        c_l = []
        for iy in np.arange(1999, 2017):
            print(iy)
            # here we suppose we only care about the combined mean of each file;
            # you might also use indexing operations like .sel to subset datasets
            combined = read_netcdfs('/Data2/SubX/RSMAS/CCSM4/'+ivar+'/'+np.str(iy)+'/*/*.nc', dim=['S','M'],
                                    transform_func=pre_process_func)
            
            c_l.append(combined)
    all_values = xr.concat(c_l, dim='S') 
    var_l.append(all_values)
all_variables = xr.merge(var_l)

tas_2m
1999


/local/ERAin/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  This is separate from the ipykernel package so we can avoid doing imports until


2000


/local/ERAin/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  This is separate from the ipykernel package so we can avoid doing imports until


2001


...

/local/ERAin/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  This is separate from the ipykernel package so we can avoid doing imports until


2010


/local/ERAin/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  This is separate from the ipykernel package so we can avoid doing imports until


2011


/local/ERAin/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
all_variables